In [1]:
%matplotlib inline


# Object Detection From TF2 Checkpoint


In [1]:
import os
import glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR') 

In [2]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Load the model

In [ ]:
!pip install object_detection

In [3]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [4]:
PATH_TO_CFG = "/home/user/Documents/CV/Low-Light-Object-Detection/ssd_efficientdet_d1_640x640_15k_ckpt/pipeline.config"
PATH_TO_CKPT = "/home/user/Documents/CV/Low-Light-Object-Detection/ssd_efficientdet_d1_640x640_15k_ckpt"

In [5]:

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 4.644453287124634 seconds


In [6]:
IMAGE_PATHS = glob.glob('/home/user/Documents/CV/Low-Light-Object-Detection/test_images/*.jpg')

PATH_TO_LABELS = '/home/user/Documents/CV/Low-Light-Object-Detection/data/label_map.txt'

SAVE_PATH = '/home/user/Documents/CV/Low-Light-Object-Detection/detections/'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)


In [7]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

In [ ]:
def load_image_into_numpy_array(path):
    
    fixed_height = 640
    image = Image.open(path)
    height_percent = (fixed_height / float(image.size[1]))
    width_size = int((float(image.size[0]) * float(height_percent)))
    image = image.resize((width_size, fixed_height), Image.NEAREST)
    return np.array(image)


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')
    
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

#     plt.figure()
#     plt.imshow(image_np_with_detections)
    if not os.path.exists(SAVE_PATH):
        os.mkdir(SAVE_PATH)
    image = Image.fromarray(image_np_with_detections)
    image.save(SAVE_PATH + '/' + image_path.split('/')[-1])
    print('Done')
# plt.show()
# sphinx_gallery_thumbnail_number = 2

In [9]:
background_img = Image.fromarray(np.zeros((640, 1000, 3), dtype='uint8'))

In [10]:
imgs = [Image.open(f) for f in sorted(glob.glob('/home/user/Documents/CV/Low-Light-Object-Detection/detections/*.jpg'))]
background_img.save(fp='detections1.gif', format='GIF', append_images=imgs,
         save_all=True, duration=1000, loop=0, disposal=3)